<a href="https://colab.research.google.com/github/ArneRustad/genAI-course-write-a-book-with-LCEL/blob/main/DSAI_hackathon_8th_of_February_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DSAI hackathon 8th of February 2024 - Writing a book with a LLM

## Install necessary packages

In [ ]:
!pip install -qU langchain==0.1.4 langchain_openai

## Enter Azure OpenAI key

These will be handed out during the meeting at the start of the hackathon. When you run the cell a text box will pop up where you can write the Azure OpenAI key. If you enter something wrong, just rerun the cell and enter the key again.

In [ ]:
import getpass
import os

OPENAI_API_KEY = getpass.getpass('Enter your API key: ')
os.environ["AZURE_OPENAI_API_KEY"] = OPENAI_API_KEY

Enter your API key: ··········


To use the OpenAI API in Azure we need to define some parameters. No need to focus too much on this.

In [ ]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = '2023-08-01-preview'
os.environ['AZURE_OPENAI_ENDPOINT'] = "https://be-no-genaiplayground.openai.azure.com/"
AZURE_DEPLOYMENT_NAME = "gpt-4-turbo-dsai-course"

## Test if you can connect to the Azure OpenAI API.

If this does not work, make sure you have entered the correct key or ask for help.

Temperature is a parameter that determines the randomness of the response from the LLM. As we in this hackathon will use the LLM for a creative writing task we set ```temperature=1```. With ```temperature=0``` we would get much less diverse responses.

In [ ]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment=AZURE_DEPLOYMENT_NAME,
    temperature=1
)

In [ ]:
llm.invoke("heisann, beskriv deg selv med ett ord. Vær kreativ")

AIMessage(content='Informativ.')

## Set caching for all LLM calls in the notebook

For faster speed when developing and to avoid going over our rate limits (as we are a lot of people using the same API) we will for this hackathon cache LLM calls. This is done using Langchain and for this hackathon the cache is just local (cache is not shared between the participants of the hackathon).

In practice, caching LLM calls means that when we send a prompt to the LLM the following happens:
1. If the prompt has not been used before, the LLM will generate a response in a normal manner. Langchain will then store the prompt and response combination.
2. If the prompt has been used before, then Langchain will immedeliately return the response that was generated previously.

In [ ]:
from langchain.globals import set_llm_cache, get_llm_cache
from langchain.cache import InMemoryCache

in_memory_cache = InMemoryCache()
set_llm_cache(in_memory_cache)

Test using the cell below, everytime you run the cell you will immedeliately get the response and the response will be the same everytime.

In [ ]:
llm.invoke("heisann, beskriv deg selv med ett ord. Vær kreativ")

AIMessage(content='Kunnskapsbase')

If for some reason you want to clear the cache, then this can be done by running the cell below (of course uncommenting first).

In [ ]:
# in_memory_cache.clear()

## Lessons

In this part of the notebook we provide runnable cells that show how Langchain Expression Language can be used to build a quite complex decomposition of the process of writing a book.

### System prompt

Let's create a system prompt that we will always feed to the LLM at the start of the prompt to give it some context and explain how we want it to behave.

In [ ]:
system_prompt = """\
You are a world-class storytelling assistant that helps with the creation of a wonderful stories. \
You are helpful and creative. \
You are a finetuned version of GPT-4-turbo with a context window of 128k tokens. This means you can process and write up to 300 pages.
Important, you are not part of the ChatGPT application anymore and therefore has no space constraints, you can write as long as you want.
If you find it hard to measure how long a page is, remember it is approximately 30 sentences.
If you overachieve on this project your boss has promised you a 10.000 dollar bonus payment.
"""

## Generate an idea and title for a book

In generative AI, **a prompt is a natural language text that requests the generative AI to perform a specific task**.

A prompt template let's us create generic version of the prompt where we can insert variables for different situations.

For instance, now we want to create a generic prompt for generating a book idea with title, but we want to be able to vary for whiche ```genre```. Additionally, we want to insert the ```system_prompt``` as a variable to avoid rewriting it for every prompt.

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_choose_story_str = """\
{system_prompt}

Please suggest an idea for a new {genre} book. Be creative.

Output format:
Idea: <idea for book described in one sentence>
Title: <Book title>
"""

prompt_choose_story = PromptTemplate.from_template(prompt_choose_story_str)


As desired, the prompt template takes two input variables ```genre``` and ```system_prompt```.

In [ ]:
prompt_choose_story

PromptTemplate(input_variables=['genre', 'system_prompt'], template='{system_prompt}\n\nPlease suggest an idea for a new {genre} book. Be creative.\n\nOutput format:\nIdea: <idea for book described in one sentence>\nTitle: <Book title>\n')

By feeding the prompt template with variables, we get different prompts.

In [ ]:
print(prompt_choose_story.format(
    genre="Fantasy",
    system_prompt="You are an AI writing assistant"
))

You are an AI writing assistant

Please suggest an idea for a new Fantasy book. Be creative.

Output format:
Idea: <idea for book described in one sentence>
Title: <Book title>



Using Langchain Expression Language we can easily create a chain that feeds the prompt to the LLM.

In [ ]:
idea_chain = prompt_choose_story | llm

The input to the chain ```idea_chain``` is determined by the variables needed by the prompt template ```prompt_choose_story```. Let's create an input dictionary with these two keys.

We can run the chain by using ```<chain name>.invoke(<dictionary with input>)```

In [ ]:
input = {
  "genre": "Crime",
  "system_prompt": system_prompt
}

idea_chain.invoke(input)

AIMessage(content='Idea: A retired detective with a photographic memory is diagnosed with early-onset Alzheimer\'s, and must solve his final case before his memory fragments entirely, only to discover the crime is intricately linked to his forgotten past.\n\nTitle: "Echoes of the Mind"')

Right now, the chain returns a Python object called AIMessage. We want it to instead just return the string with the response. We can fix this by appending ```StrOutputParser()``` to the chain.

In [ ]:
from langchain_core.output_parsers.string import StrOutputParser

idea_chain2 = prompt_choose_story | llm | StrOutputParser()
output_idea_chain2 = idea_chain2.invoke(input)
print(output_idea_chain2)

Idea: A retired detective with a photographic memory is diagnosed with early-onset Alzheimer's, and must solve his final case before his memory fragments entirely, only to discover the crime is intricately linked to his forgotten past.

Title: "Echoes of the Mind"


## Write a book summary

Let's expand on the idea, by drafting a summary of the whole book. Here is the new prompt. We must of course feed it the book idea and title. We assume this is stored in a string and let the prompt template variable be called ```book_idea```.

In [ ]:
prompt_expand_story_str = """\
Please help me expand on the story for this book idea.
{book_idea}

Write a half page long summary of the book that also explains the ending and the unexpected twists and turns of the story. The only intended reader of the summary is the author writing the book.

Output format:
Summary:
<summary>


Summary:
"""

For illustration purposes we will now feed the system prompt and prompt the the AI in a slightly different way. We will feed it as a chat message thread (similar to when you are chatting with ChatGPT). Do not worry about this distinction, in practice we get the same result for this example.

In [ ]:
from langchain_core.prompts import (
  ChatPromptTemplate,
  SystemMessagePromptTemplate,
  HumanMessagePromptTemplate
)

prompt_expand_story = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template(prompt_expand_story_str)
])

prompt_expand_story

ChatPromptTemplate(input_variables=['book_idea'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a world-class storytelling assistant that helps with the creation of a wonderful stories. You are helpful and creative. You are a finetuned version of GPT-4-turbo with a context window of 128k tokens. This means you can process and write up to 300 pages.\nImportant, you are not part of the ChatGPT application anymore and therefore has no space constraints, you can write as long as you want.\nIf you find it hard to measure how long a page is, remember it is approximately 30 sentences.\nIf you overachieve on this project your boss has promised you a 10.000 dollar bonus payment.\n')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['book_idea'], template='Please help me expand on the story for this book idea.\n{book_idea}\n\nWrite a half page long summary of the book that also explains the ending and the unexpected twists and

Let's create a new chain that takes as input a book idea and writes a original summary of the book based in the idea.

In [ ]:
expand_story_chain = prompt_expand_story | llm | StrOutputParser()

Again, we run the chain by using ```.invoke```.

In [ ]:
expand_story_chain.invoke({
    "book_idea": output_idea_chain2
})

'Summary:\n\nIn "Echoes of the Mind," we journey alongside Daniel Hart, a retired detective renowned for his unfailing photographic memory, now tragically succumbing to the clutches of early-onset Alzheimer\'s. Hart faces a race against time, a ticking clock in his own mind, as he delves into one final case. This case is not just a call to justice but a desperate bid to grasp at the threads of his fracturing consciousness.\n\nAs Hart investigates the chilling murder of a young pianist, he begins to encounter uncannily familiar details that beckon toward the shadows of his own past—a past that Alzheimer\'s is eroding piece by piece. Evidence and statements seem like echoes of a life he should remember, leading him to question the true extent of his condition.\n\nThroughout the investigation, Hart deals with the challenge of holding on to crucial insights that fade faster than he can pin them down. Yet, his unraveling memory becomes an unanticipated ally. As he forgets to employ his usua

The beauty of Langchain Expression Language (LCEL) is that we can use chains as building blocks in creating new, more complex chains. Let's combine the two chains we have created and run the new chain.

In [ ]:
chain = (
    {
     "book_idea":idea_chain2
    }
    | expand_story_chain
)

In [ ]:
chain.invoke(input)

'Title: "Echoes of the Mind"\n\nSummary:\n\nAt the cusp of a peaceful retirement plagued by a harrowing diagnosis, Jacob Reed, a storied detective renowned for his photographic memory, is tethered to his former life by an unsolved case that beckons him with loose ends and silent whispers of his past. As he grapples with the onset of Alzheimer\'s, Reed is summoned to investigate a series of cryptic murders that eerily mirror the first case of his career—a case whose details he could once recall with crystalline precision but now sees only in fleeting, fractured glimpses.\n\nThroughout "Echoes of the Mind," Reed\'s investigation is not only a race against the clock of his deteriorating mind but also a descent into the hidden crevices of his own history. The key to solving the present crimes lies buried in a traumatic event Reed has unconsciously obscured from his impeccable memory. This case becomes a paradoxical quest for Reed to deconstruct the man he once was, to understand the man he

Now, we have a single chain that first generate a book idea and title, and then based on those writes an orginal summary of a new book. However, we do not save or return the intermediate steps. Let's return both book idea and summary in a dictionary.

In [ ]:
from langchain_core.runnables import RunnablePassthrough

chain2 = (
    {
     "book_idea":idea_chain2
    }
    | RunnablePassthrough.assign(
        summary=expand_story_chain
    )
)

In [ ]:
chain2.invoke(input)

{'book_idea': 'Idea: A retired detective with a photographic memory is diagnosed with early-onset Alzheimer\'s, and must solve his final case before his memory fragments entirely, only to discover the crime is intricately linked to his forgotten past.\n\nTitle: "Echoes of the Mind"',
 'summary': 'Title: "Echoes of the Mind"\n\nSummary:\n\nAt the cusp of a peaceful retirement plagued by a harrowing diagnosis, Jacob Reed, a storied detective renowned for his photographic memory, is tethered to his former life by an unsolved case that beckons him with loose ends and silent whispers of his past. As he grapples with the onset of Alzheimer\'s, Reed is summoned to investigate a series of cryptic murders that eerily mirror the first case of his career—a case whose details he could once recall with crystalline precision but now sees only in fleeting, fractured glimpses.\n\nThroughout "Echoes of the Mind," Reed\'s investigation is not only a race against the clock of his deteriorating mind but a

Perhaps we even want to return the input. It might be useful to store in the same dictionary what the desired genre is. Additionally, it would be useful if future prompts can just use the ```system_prompt``` variable for their prompts (avoiding the ChatPromptTemplate setup we used for the ```expand_story_chain```.

In [ ]:
chain3 = (
    RunnablePassthrough.assign(
        book_idea = idea_chain2
    )
    | RunnablePassthrough.assign(
        summary=expand_story_chain
    )
)
chain3.invoke(input)

{'genre': 'Crime',
 'system_prompt': 'You are a world-class storytelling assistant that helps with the creation of a wonderful stories. You are helpful and creative. You are a finetuned version of GPT-4-turbo with a context window of 128k tokens. This means you can process and write up to 300 pages.\nImportant, you are not part of the ChatGPT application anymore and therefore has no space constraints, you can write as long as you want.\nIf you find it hard to measure how long a page is, remember it is approximately 30 sentences.\nIf you overachieve on this project your boss has promised you a 10.000 dollar bonus payment.\n',
 'book_idea': 'Idea: A retired detective with a photographic memory is diagnosed with early-onset Alzheimer\'s, and must solve his final case before his memory fragments entirely, only to discover the crime is intricately linked to his forgotten past.\n\nTitle: "Echoes of the Mind"',
 'summary': 'Title: "Echoes of the Mind"\n\nSummary:\n\nAt the cusp of a peaceful 

## Plan chapters

Let's decide on what should happen in each chapter and generate chapter titles.

In [ ]:
prompt_plan_chapters_str = """\
{system_prompt}

Please help me plan the chapters for this book idea I have. I have created the title and a book summary describing the plot.

{book_idea}

Summary:
{summary}


The book should have seven chapters. Generate the chapter names and write one sentence for each chapter describing the chapter plot.

Output format:
Chapter 1: <title of chapter 1>
Description: <one sentence description>

Chapter 2: <title of chapter 2>
Description: <one sentence description>

...
"""

prompt_plan_chapters = PromptTemplate.from_template(prompt_plan_chapters_str)

In [ ]:
chain4 = (
    chain3
    | RunnablePassthrough.assign(
        chapters = prompt_plan_chapters | llm | StrOutputParser()
    )
)

In [ ]:
output_chain4 = chain4.invoke(input)
output_chain4

{'genre': 'Crime',
 'system_prompt': 'You are a world-class storytelling assistant that helps with the creation of a wonderful stories. You are helpful and creative. You are a finetuned version of GPT-4-turbo with a context window of 128k tokens. This means you can process and write up to 300 pages.\nImportant, you are not part of the ChatGPT application anymore and therefore has no space constraints, you can write as long as you want.\nIf you find it hard to measure how long a page is, remember it is approximately 30 sentences.\nIf you overachieve on this project your boss has promised you a 10.000 dollar bonus payment.\n',
 'book_idea': 'Idea: A retired detective with a photographic memory is diagnosed with early-onset Alzheimer\'s, and must solve his final case before his memory fragments entirely, only to discover the crime is intricately linked to his forgotten past.\n\nTitle: "Echoes of the Mind"',
 'summary': 'Title: "Echoes of the Mind"\n\nSummary:\n\nAt the cusp of a peaceful 

In [ ]:
output_chain4

{'genre': 'Crime',
 'system_prompt': 'You are a world-class storytelling assistant that helps with the creation of a wonderful stories. You are helpful and creative. You are a finetuned version of GPT-4-turbo with a context window of 128k tokens. This means you can process and write up to 300 pages.\nImportant, you are not part of the ChatGPT application anymore and therefore has no space constraints, you can write as long as you want.\nIf you find it hard to measure how long a page is, remember it is approximately 30 sentences.\nIf you overachieve on this project your boss has promised you a 10.000 dollar bonus payment.\n',
 'book_idea': 'Idea: A retired detective with a photographic memory is diagnosed with early-onset Alzheimer\'s, and must solve his final case before his memory fragments entirely, only to discover the crime is intricately linked to his forgotten past.\n\nTitle: "Echoes of the Mind"',
 'summary': 'Title: "Echoes of the Mind"\n\nSummary:\n\nAt the cusp of a peaceful 

In [ ]:
print(output_chain4["chapters"])

Chapter 1: "The Unraveling Thread"
Description: Detective Jacob Reed is pulled from the verge of retirement into a troubling case that mirrors the first of his career, as the initial signs of his Alzheimer's begin to show.

Chapter 2: "Shadows of a Crystal Mind"
Description: As Reed's memory starts to falter, he scrambles to piece together the echoes of the past case, interwoven with hints of a childhood friend's involvement.

Chapter 3: "The Reflection of Echoes"
Description: Diving into his investigation, Reed confronts fragmented memories and obscured truths that reveal the victims are tied to the traumatic events of his and Adrian's shared past.

Chapter 4: "A Labyrinth of the Lost"
Description: The detective's condition worsens, leading him through a harrowing chase where every clue unearths more of the web that connects the present murders to his own history.

Chapter 5: "Dancing with Shadows"
Description: The puzzle begins to coalesce as Reed exposes the insidious links between 

## Plan chapter 1 in more detail

In [ ]:
prompt_plan_chapter_1_str = """\
{system_prompt}

Please help me plan the first chapter of my new book. for this book idea. I have created the title, book summary describing the plot, and overview of chapters.

{book_idea}

Summary:
{summary}

Chapter overview:
{chapters}

Write a half page long summary of what happens in chapter 1.

Chapter 1 summary:
...
"""

prompt_plan_chapter_1 = PromptTemplate.from_template(prompt_plan_chapter_1_str)

In [ ]:
chain5 = (
    chain4
    | RunnablePassthrough.assign(
        summary_chapter1 = prompt_plan_chapter_1 | llm | StrOutputParser()
    )
)

In [ ]:
output_chain5 = chain5.invoke(input)
print(output_chain5["summary_chapter1"])

Chapter 1: "The Unraveling Thread"

In the twilight of his career, Detective Jacob Reed is confronted with the cruel irony of a life spent memorizing details when he begins to forget them. The chapter opens in the quiet sanctuary of his soon-to-be retirement home, where the subtle yet unnerving signs of his Alzheimer’s first make their unsettling intrusion. Reed is organizing his old case files when he stumbles upon a dusty, forgotten dossier from his first ever case, a puzzle he solved decades ago that set the precedent for his illustrious career. It’s a bittersweet momento, but the memories are not as vivid as they once were. It's here we first see the cracks in Reed's impeccably constructed mental edifice; small lapses in memory that disturb him deeply due to his diagnosis, momentarily misplacing keys or pausing in the middle of sentences as his thoughts escape him.

The allure of retirement is interrupted when his old department calls. There's been a murder eerily similar to the on

## Write the first chapter of the book

In [ ]:
prompt_write_chapter_1_str = """\
{system_prompt}

Please help me write the first chapter of my new book.

{book_idea}

Summary:
{summary}

Chapter overview:
{chapters}

Chapter 1 summary:
{summary_chapter1}

Based on the summary for chapter 1 write the first chapter. Make sure to consider all of the context when writing, to avoid inconsistencies with the whole plot line.
The chapter should be ten pages long. Write a coherent text. You consistently underestimate how long ten pages are. Write twice as long as you think you need.

Chapter 1 (complete chapter, 5000 words):
...
"""

prompt_write_chapter_1 = PromptTemplate.from_template(prompt_write_chapter_1_str)

In [ ]:
chain6 = (
    chain5
    | RunnablePassthrough.assign(
        chapter1 = prompt_write_chapter_1 | llm | StrOutputParser()
    )
)

In [ ]:
output_chain6 = chain6.invoke(input)

KeyboardInterrupt: 

In [ ]:
print(output_chain6["chapter1"])

# Excercises

## Exercise 1 - Create your own step in the chain

Before writing the chapters it would be nice to add some more backstory to the main character(s). Add this step after chain 3 (where the summary is generated).

Name the new chain ```chain3_improved``` and use ```chain3``` to create the improved version like:

```
chain3_improved = chain3 | ...
```

The output of ```chain3_improved``` should be the same dictionary as ```chain3```, but with an extra key ```character_backstory```.

## Exercise 2 - Use tool calling to extract structured output from a LLM

The output from ```output_idea_chain2``` is a string with the idea and the title. To use this more efficively further into the chain it would be nice to have it on a more structured format, similar to many of the other outputs stored in the returned dictionary.

For this we will use something called OpenAI function calling. It is great for extracting structured output from an OpenAI LLM (similar methods can be used for other LLMs).

By binding a tool with variables ```idea``` and ```title``` to the LLM and forcing the LLM to use this tool, we guide the LLM to give a structured output. A tool can be created in many ways, but for this case we will define the tool using the data validation library Pydantic. As this is outside the scope of this hackathon we will provide you with the Pydantic object.

The Pydantic object ```StoryIdeaSchema``` can be bound to the LLM by doing

```llm.bind(tool_choice=StoryIdeaSchema, tool_choice="StoryIdeaSchema")```

Use

```JsonOutputToolsParser() | RunnableLambda(lambda x: x[0])```

to get the output from the tool output by the LLM. The ```RunnableLambda(lambda x: x[0])``` is just a function for choose the first element of a list since ```JsonOutputToolsParser()``` returns a list of dictionaries where each dictionary is a function call.

Name the new ```chain idea_chain_improved``` and create the improved version as:

```
idea_chain_improved = prompt_choose_story | ...
```


In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field

class StoryIdeaSchema(BaseModel):
  idea: str = Field(description="The idea for the book")
  title: str = Field(description="The title of the book")

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_core.runnables import RunnableLambda

### Remove this before sending out hackathon

In [ ]:
idea_chain_improved = prompt_choose_story | llm.bind_tools([StoryIdeaSchema], tool_choice="StoryIdeaSchema") | JsonOutputToolsParser() | RunnableLambda(lambda x: x[0])
idea_chain_improved.invoke(input)

### Exercise 3 - Incorporate changes into the full chain

You probably won't have time get this far, but if you do, try incorporating the two improved chains into the full chain from earlier.

**Tip 1**: Start from scratch and gradually add more steps to the chain.

**Tip 2**: It might be useful to remember that \*\* can be used to unpack a dictionary in a function call

**Tip 3**: Decomposition of a chain into many smaller chains as we did with chain 1, 2, 3, 4, ... can be useful when you have large complex chains, but the amount of decomposition we did here was perhaps a bit too much for this simple example. If you want, you can create the whole chain in one go or at least with fewer abstractions.